In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys

sys.path.append('./dashboard')

In [3]:
from typing import List, Union, Literal, TypedDict, Optional
import os

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.exceptions import OutputParserException
from langchain_anthropic import ChatAnthropic
from langchain_community.callbacks import get_openai_callback

In [4]:
from pydantic import (
    BaseModel,
    ValidationError,
    ValidationInfo,
    field_validator,
)
from pydantic.v1 import BaseModel as BaseModelV1
from pydantic.v1 import create_model, validator, ValidationError

In [5]:
from dashboard.model import get_model
from dashboard.plan import get_dashboard_plan

In [6]:
import vizro.plotly.express as px
import vizro.models as vm
import pandas as pd
from functools import partial

## OpenAI

In [8]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

## Anthropic

In [8]:
# import os
# from anthropic import Anthropic

# client = Anthropic(
#     # This is the default and can be omitted
#     api_key=os.environ.get("ANTHROPIC_API_KEY"),
#     base_url=os.environ.get("ANTHROPIC_BASE_URL")
# )

# message = client.messages.create(
#     max_tokens=1024,
#     messages=[
#         {
#             "role": "user",
#             "content": "Hello, Claude",
#         }
#     ],
#     model="claude-3-opus-20240229",
# )
# print(message.content)

## Choose model

In [9]:
model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# model = ChatAnthropic(
#     model='claude-3-opus-20240229',
#     anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"),
#     anthropic_api_url=os.environ.get("ANTHROPIC_BASE_URL")
# )

## Starting to construct a page

In [10]:
query = ("I need a page with a bar chart shoing the population per continent "
    "and a scatter chart showing the life expectency per country as a function gdp. "
    "Make a filter on the GDP column and use a dropdown as selector. This filter should only "
    "apply to the bar chart. The bar chart should be a stacked bar chart, while "
    "the scatter chart should be colored by the column 'continent'. I also want "
    "a table that shows the data. The title of the page should be `My wonderful "
    "jolly dashboard showing a lot of data`. I also want a second page with just "
    "a card on it that links to `https://vizro.readthedocs.io/`")

In [11]:
df = px.data.gapminder()

In [12]:
dashboard_plan = get_dashboard_plan(query, model)

In [13]:
dashboard_plan

DashboardPlanner(pages=[PagePlanner(title='My wonderful jolly dashboard showing a lot of data', components=Components(components=[Component(component_name='Graph', component_description='A stacked bar chart showing the population per continent.', component_id='population_bar_chart'), Component(component_name='Graph', component_description='A scatter chart showing life expectancy per country as a function of GDP, colored by continent.', component_id='life_expectancy_scatter_chart'), Component(component_name='AgGrid', component_description='A table displaying the data used in the charts.', component_id='data_table')]), controls=Controls(controls=[Control(control_name='Filter', control_description='A dropdown selector to filter data based on GDP values. This filter applies only to the bar chart.')])), PagePlanner(title='Link to Documentation', components=Components(components=[Component(component_name='Card', component_description='A card linking to the Vizro documentation at https://vizr

In [14]:
page_plan = dashboard_plan.pages[0]

In [15]:
dashboard_plan.pages[1]

PagePlanner(title='Link to Documentation', components=Components(components=[Component(component_name='Card', component_description='A card linking to the Vizro documentation at https://vizro.readthedocs.io/', component_id='vizro_link_card')]), controls=Controls(controls=[]))

In [16]:
control_plan = page_plan.controls.controls[0]

In [17]:
control_plan

Control(control_name='Filter', control_description='A dropdown selector to filter data based on GDP values. This filter applies only to the bar chart.')

In [18]:
available_components = [comp.component_id for comp in page_plan.components.components]
available_components

['population_bar_chart', 'life_expectancy_scatter_chart', 'data_table']

In [23]:
vm.Graph(id = "population_bar_chart",figure=px.bar(df,x = "country"))

Graph(id='population_bar_chart', type='graph', figure=<vizro.models.types.CapturedCallable object at 0x7fc1e86e2d90>, actions=[])

In [19]:


control_plan.create(df=df,model=model,available_components=available_components)

/Users/Maximilian_Schulz/Library/Application Support/hatch/env/virtual/vizro-ai/fZiNbsWT/vizro-ai/lib/python3.9/site-packages/pydantic/v1/schema.py:631: UserWarning:

CapturedCallable function is excluded from the schema.



Filter(id='e443df78-9558-867f-5ba9-1faf7a024204', type='filter', column='gdpPercap', targets=['population_bar_chart'], selector=Dropdown(id='23a7711a-8133-2876-37eb-dcd9e87a1613', type='dropdown', options=[], value=None, multi=True, title='Select GDP', actions=[ActionsChain(id='1846d424-c17c-6279-23c6-612f48268673', trigger=Trigger(component_id='23a7711a-8133-2876-37eb-dcd9e87a1613', component_property='value'), actions=[])]))

In [20]:
# class PageBuilder:
#     def __init__(self, model,page_plan):
#         self.model = model
#         self.page_plan = page_plan

#     def _build_components():
    